<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/TreePedia_Step2_checking_gsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To access the google streetview API, one should have the API key.**
* First, get started with Google Maps platform: https://developers.google.com/maps/gmp-get-started
* Then, get an API key to use Street View Static API: https://developers.google.com/maps/documentation/streetview/get-api-key

---
This notebook uses google-streetview package: https://pypi.org/project/google-streetview/. 
* For documentation, please see https://rrwen.github.io/google_streetview/



In [30]:
!pip install google-streetview

In [42]:
import pandas as pd
import google_streetview.api
import math

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!ls drive/My\ Drive/StreetPsych

Chicago  HydePark  key.txt


In [45]:
# Google Street View Static API key
with open('drive/My Drive/StreetPsych/key.txt') as file:
  api_key = file.read()

In [46]:
#location = 'HydePark'
location = 'Chicago'

picked_df = pd.read_csv('drive/My Drive/StreetPsych/' + location + '/seed_' + location + '.csv')
picked_df

,Unnamed: 0,geometry,bearing,osm_index,lat,lng
0,0,POINT (-87.72991748918686 41.77217893981616),358.898541,6,41.772179,-87.729917
1,1,POINT (-87.72993107476934 41.7727059262377),358.898550,6,41.772706,-87.729931
2,2,POINT (-87.59195064934102 41.78966068712851),269.222074,34,41.789661,-87.591951
3,3,POINT (-87.74564239790189 41.95554006274232),358.747691,55,41.955540,-87.745642
4,4,POINT (-87.75108523885028 41.95793764774632),269.015632,58,41.957938,-87.751085
...,...,...,...,...,...,...
47011,47011,POINT (-87.69278144322213 41.84181049656495),187.404732,187397,41.841810,-87.692781
47012,47012,POINT (-87.6375455164422 41.85210445538159),238.761882,187484,41.852104,-87.637546
47013,47013,POINT (-87.63766551188191 41.85245944454397),324.421404,187484,41.852459,-87.637666
47014,47014,POINT (-87.68365475100006 41.92417791019385),269.223080,187505,41.924178,-87.683655


In [47]:
params = []
for row in picked_df.head(20).iterrows():
  params.append({
            'key': api_key,
            'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
  })
#'size': '600x480', # max 640x640 pixels
#print(params)

In [48]:
results = google_streetview.api.results(params)
#results.preview()
len(results.metadata)

20

In [49]:
# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrame
# purpose - splits the DataFrame into smaller chunks
def split_dataframe(df, chunk_size = 200): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def generate_param(df, api_key):
  params = []
  for row in df.iterrows():
    params.append({
              'key': api_key,
              'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
    })
  return params

In [ ]:
#split_dataframe(picked_df)[1][['lat','lng']].reset_index(drop=True)

In [50]:
results_df = pd.DataFrame()
split_dfs = split_dataframe(picked_df)
count = 1

for df in split_dfs:
  print('Processing ', count, ' / ', len(split_dfs))
  results = google_streetview.api.results(generate_param(df, api_key))
  gsv_info = []
  for row in results.metadata:
    if 'OK' in row['status']:
      if 'Google' in row['copyright']:
        gsv_info.append([row['location']['lat'], row['location']['lng'], row['pano_id'], row['date'], row['status']])
      else:
        gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])
    else:
      gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])

  results_df = results_df.append( pd.DataFrame(gsv_info, columns = ['gsv_lat', 'gsv_lng', 'pano_id', 'date', 'status']) )
  count += 1

print('Done.')  


Processing  1  /  236
Processing  2  /  236
Processing  3  /  236
Processing  4  /  236
Processing  5  /  236
Processing  6  /  236
Processing  7  /  236
Processing  8  /  236
Processing  9  /  236
Processing  10  /  236
Processing  11  /  236
Processing  12  /  236
Processing  13  /  236
Processing  14  /  236
Processing  15  /  236
Processing  16  /  236
Processing  17  /  236
Processing  18  /  236
Processing  19  /  236
Processing  20  /  236
Processing  21  /  236
Processing  22  /  236
Processing  23  /  236
Processing  24  /  236
Processing  25  /  236
Processing  26  /  236
Processing  27  /  236
Processing  28  /  236
Processing  29  /  236
Processing  30  /  236
Processing  31  /  236
Processing  32  /  236
Processing  33  /  236
Processing  34  /  236
Processing  35  /  236
Processing  36  /  236
Processing  37  /  236
Processing  38  /  236
Processing  39  /  236
Processing  40  /  236
Processing  41  /  236
Processing  42  /  236
Processing  43  /  236
Processing  44  /  2

In [51]:
combined_df = picked_df[['lat','lng','bearing','osm_index']].merge(results_df.reset_index(drop=True), left_index=True, right_index=True, suffixes=(False, False))
combined_df  

,lat,lng,bearing,osm_index,gsv_lat,gsv_lng,pano_id,date,status
0,41.772179,-87.729917,358.898541,6,41.772188,-87.729909,nT-cvjnNrRGc7i-lhngV3w,2019-09,OK
1,41.772706,-87.729931,358.898550,6,41.772673,-87.729925,TOoMmprVSSoCDq-UldfJ3g,2019-09,OK
2,41.789661,-87.591951,269.222074,34,41.789655,-87.591927,3wsVSDUKXIuc_h47TK2FNw,2019-04,OK
3,41.955540,-87.745642,358.747691,55,41.955535,-87.745638,8-KAUjdGCVU2840MRBqKOw,2019-06,OK
4,41.957938,-87.751085,269.015632,58,41.957933,-87.751063,qDg4oig_lVxiufBPoRoyIg,2019-06,OK
...,...,...,...,...,...,...,...,...,...
47011,41.841810,-87.692781,187.404732,187397,41.841852,-87.692765,n1SGBHPKlHoUzmYsYIRvpA,2019-07,OK
47012,41.852104,-87.637546,238.761882,187484,NaN,NaN,NaN,NaN,NaN
47013,41.852459,-87.637666,324.421404,187484,41.852825,-87.637691,xpXOKDR-YCmAlFq0SX_VAw,2019-08,OK
47014,41.924178,-87.683655,269.223080,187505,41.924176,-87.683697,my81qGYG4daMaTQvrwYwJA,2019-07,OK


In [52]:
combined_df.to_csv('drive/My Drive/StreetPsych/' + location + '/gsv_seed_' + location + '.csv')